In [1]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import numpy as np

In [2]:
tf.__version__

'2.9.1'

# Create a Submission File 

The submitted file must include all the predictions from Dan, Kyu, and Play Style testing CSVs. Therefore, you must train your models before submitting any file. If your model is not trained for Dan, Kyu, or Play Style, check out the previous tutorials about how to train your model or just simply use the baseline models (model_dan_tutorial.h5, model_kyu_tutorial.h5, model_playstyle.h5) 

**PublicUpload.csv** must be in the following form:
```
DTPU0000000001,id,qr,pq,pd,ab
DTPU0000000002,ao,ab,ha,ff,qd
DTPU0000000003,qd,gd,fh,ed,fa
DTPU0000000004,pr,ba,dq,hh,aj
DTPU0000000005,ph,jh,af,df,gj
.
.
.
KTPU0000000001,id,qr,pq,pd,ab
KTPU0000000002,ao,ab,ha,ff,qd
KTPU0000000003,qd,gd,fh,ed,fa
KTPU0000000004,pr,ba,dq,hh,aj
KTPU0000000005,ph,jh,af,df,gj
.
.
.
PSTPU0000000001_79,2
PSTPU0000000001_55,3
PSTPU0000000001_105,2
PSTPU0000000002_37,1
PSTPU0000000002_113,3
```

**For Dan and Kyu:**
- Column 1: Game ID
- Column 2: Predicted Moves, up to 5 predictions for each game

**For PlayStyle**:
- Column 1: Game ID
- Column 2: Predicted Game Style

The code block below is copied from the tutorial notebooks and it includes the pre-processing methods of the trained models. You might need to change the functions below with respect to your models.

In [3]:
chars = 'abcdefghijklmnopqrs'
coordinates = {k:v for v,k in enumerate(chars)}
chartonumbers = {k:v for k,v in enumerate(chars)}

def prepare_input_for_dan_kyu_models(moves, player_color):
    x = np.zeros((19,19,13))
#     player_color = moves[-1][0] #
    for move in moves:
        color = move[0]
        column = coordinates[move[2]]
        row = coordinates[move[3]]
        if color == player_color: #
            x[row,column,0] = 1
            x[row,column,2] = 1
        else: #
            x[row,column,1] = 1
            x[row,column,2] = 1
    x[:,:,2] = np.where(x[:,:,2] == 0, 1, 0)
    
    #倒數8步
    sz = len(moves)
    last1 = sz - 1
    last8 = max(sz - 9, 0)
    for i in range(last1, last8, -1):
        col = coordinates[moves[i][2]]
        row = coordinates[moves[i][3]]
        x[row, col, 11 - (sz - i)] = 1
    
    #周圍7*7
    last_col = coordinates[moves[-1][2]]
    last_row = coordinates[moves[-1][3]]
    rad = 3 #要改範圍大小的話改這個
    row1 = max(0, last_row - rad)
    row7 = min(18, last_row + rad)
    col1 = max(0, last_col - rad)
    col7 = min(18, last_col + rad)
    for i in range(row1, row7 + 1, 1):
        for j in range(col1, col7 + 1, 1):
            x[i, j, 11] = x[i, j, 0]
            x[i, j, 12] = x[i, j, 1]
            
    #列印所有棋盤
#     for i in range(0, 3, 1):
#         print("  a b c d e f g h i j k l m n o p q r s")
#         for j in range(0, 19, 1):
#             print(chars[j], end = " ")
#             for k in range(0, 19, 1):
#                 print(int(x[j, k, i]), end = " ")
#             print("")
#         print("")
    
    return x

def prepare_input_for_playstyle_model(moves):
    x = np.zeros((19,19,14))
    player_color = moves[-1][0] #
    for move in moves:
        color = move[0]
        column = coordinates[move[2]]
        row = coordinates[move[3]]
        if color == player_color: #
            x[row,column,0] = 1
            x[row,column,2] = 1
        else: #
            x[row,column,1] = 1
            x[row,column,2] = 1
    x[:,:,2] = np.where(x[:,:,2] == 0, 1, 0)
    
    #倒數8步
    sz = len(moves)
    last1 = sz - 1
    last8 = max(sz - 9, 0)
    for i in range(last1, last8, -1):
        col = coordinates[moves[i][2]]
        row = coordinates[moves[i][3]]
        x[row, col, 11 - (sz - i)] = 1
    
    #周圍7*7
    last_col = coordinates[moves[-1][2]]
    last_row = coordinates[moves[-1][3]]
    rad = 3 #要改範圍大小的話改這個
    row1 = max(0, last_row - rad)
    row7 = min(18, last_row + rad)
    col1 = max(0, last_col - rad)
    col7 = min(18, last_col + rad)
    for i in range(row1, row7 + 1, 1):
        for j in range(col1, col7 + 1, 1):
            x[i, j, 11] = x[i, j, 0]
            x[i, j, 12] = x[i, j, 1]
            x[i, j, 13] = x[i, j, 2]
    #列印所有棋盤
#     for i in range(0, 3, 1):
#         print("  a b c d e f g h i j k l m n o p q r s")
#         for j in range(0, 19, 1):
#             print(chars[j], end = " ")
#             for k in range(0, 19, 1):
#                 print(int(x[j, k, i]), end = " ")
#             print("")
#         print("")
    
    return x

def number_to_char(number):
    number_1, number_2 = divmod(number, 19)
    return chartonumbers[number_1] + chartonumbers[number_2]

def top_5_preds_with_chars(predictions):
    resulting_preds_numbers = [np.flip(np.argpartition(prediction, -5)[-5:]) for prediction in predictions]
    resulting_preds_chars = np.vectorize(number_to_char)(resulting_preds_numbers)
    return resulting_preds_chars

## Save Predictions for Dan
The code block below is to use **dan model** to predict and save the results in required form. It generates the best 5 predictions for each sample and convert them to character coordinates.

In [12]:
# Load your own model. Here we use the baseline model
model = load_model('./model_dan_one_epoch.h5')

# Load the corresponding dataset
df = open('./CSVs/dan_test_public.csv').read().splitlines()
games_id = [i.split(',',2)[0] for i in df]
games = [i.split(',',2)[-1] for i in df]
colors = [i.split(',',2)[1] for i in df]

x_testing = []

for game_i, game in enumerate(games):
    moves_list = game.split(',')
    x_testing.append(prepare_input_for_dan_kyu_models(moves_list, colors[game_i]))

x_testing = np.array(x_testing)
predictions = model.predict(x_testing)
prediction_chars = top_5_preds_with_chars(predictions)

# Save results to public_submission_template.csv
with open('./public_submission_template.csv','a') as f:
    for index in range(len(prediction_chars)):
        answer_row = games_id[index] + ',' + ','.join(prediction_chars[index]) + '\n'
        f.write(answer_row)

ValueError: in user code:

    C:\Users\kenny\AppData\Local\anaconda3\envs\aicup_\lib\site-packages\keras\engine\training.py:1586 predict_function  *
        return step_function(self, iterator)
    C:\Users\kenny\AppData\Local\anaconda3\envs\aicup_\lib\site-packages\keras\engine\training.py:1576 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    C:\Users\kenny\AppData\Local\anaconda3\envs\aicup_\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:1286 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    C:\Users\kenny\AppData\Local\anaconda3\envs\aicup_\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2849 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    C:\Users\kenny\AppData\Local\anaconda3\envs\aicup_\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:3632 _call_for_each_replica
        return fn(*args, **kwargs)
    C:\Use

## Save Predictions for Kyu
The same code block applies to Kyu by changing the model and dataset

In [4]:
# Load your own model. Here we use the baseline model
model = load_model('./models/model_kyu_resnet_fromgogame_withnor.h5')

# Load the corresponding dataset
df = open('./CSVs/kyu_test_public.csv').read().splitlines()
games_id = [i.split(',',2)[0] for i in df]
games = [i.split(',',2)[-1] for i in df]
colors = [i.split(',',2)[1] for i in df]

x_testing = []

for game_i, game in enumerate(games):
    moves_list = game.split(',')
    x_testing.append(prepare_input_for_dan_kyu_models(moves_list, colors[game_i]))

x_testing = np.array(x_testing)
predictions = model.predict(x_testing)
prediction_chars = top_5_preds_with_chars(predictions)

# Save results to public_submission_template.csv
with open('./public_submission_template.csv','a') as f:
    for index in range(len(prediction_chars)):
        answer_row = games_id[index] + ',' + ','.join(prediction_chars[index]) + '\n'
        f.write(answer_row)

## Save Predictions for PlayStyle
Playstyle requires only one predictions so the code here is a simple version of above

In [4]:
# Load your own model. Here we use the baseline model
model = load_model('./models/model_playstyle_resnet_original.h5')

# Load the corresponding dataset
df = open('./CSVs/play_style_test_public.csv').read().splitlines()
games_id = [i.split(',',2)[0] for i in df]
games = [i.split(',',2)[-1] for i in df]

x_testing = []

for game in games:
    moves_list = game.split(',')
    x_testing.append(prepare_input_for_playstyle_model(moves_list))

x_testing = np.array(x_testing)
predictions = model.predict(x_testing)
prediction_numbers = np.argmax(predictions, axis=1)
# print(np.shape(x_testing))
with open('./public_submission_template.csv','a') as f:
    for index, number in enumerate(prediction_numbers):
        answer_row = games_id[index] + ',' + str(number+1) + '\n'
        f.write(answer_row)

313/313 [==============================] - 23s 39ms/step


Now you can upload your file from the competition website! (https://tbrain.trendmicro.com.tw/Competitions/Details/29)